OSCILADOR ARMÓNICO AMORTIGUADO FORZADO: Diagramas de Poincaré, dimensión fractal y distribución de tiempos

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy as scipy
from math import *

VALORES NUMÉRICOS DEL PROBLEMA

Condiciones iniciales

In [ ]:
x_0 = 1 # Posición inicial
v_0 = 0 # Velocidad inicial

Parámetros del sistema

In [ ]:
w_0 = 0.5 # Frecuencia natural x>0
W_0 = 100 # Frecuencia natural x<0
amort = 0.125 # Constante de amortiguamiento
f_0 = 0.25 # Amplitud de la fuerza por unidad de masa

Medida del tiempo

In [ ]:
dt = 0.001 # Paso de tiempo
tf = 5000 # Tiempo de medida (se asume que t0 = 0)
t = np.linspace(0,tf,int(tf/dt)+1) # Vector de instantes de tiempo

Frencuencia angular de la fuerza externa

In [ ]:
dw = 0.001 # paso en la frecuencia externa
w =  np.array([i*dw for i in range(int(12*0.1/dw),int(1.6/dw)+1)]) # Frecuencia fuerza externa

DEFINIMOS UNA FRECUENCIA NATURAL DEL OSCILADOR QUE DEPENDE DE LA POSICIÓN

In [ ]:
def W(x): # frecuecia angular
    return W_0 - (W_0-w_0)*np.heaviside(x,w_0)

rep = np.linspace(-5,5,5000)
plt.plot(rep,W(rep)) # Representación gráfica de la frecuencia natural
plt.title('Frencuencia natural del oscilador')
plt.xlabel('x (m)')
plt.ylabel('Ω₀ (s⁻¹)')
plt.xlim([min(rep),max(rep)])
plt.savefig('Frecuencia natural.png')
plt.show()

POTENCIAL DEL OSCILADOR

In [ ]:
U = (1/2)*W(rep)**2*rep**2
rep = np.linspace(-5,5,5000)

plt.plot(rep,U)
plt.title('Potencial del oscilador')
plt.xlabel('x (m)')
plt.ylabel('U (J)')
plt.xlim([min(rep),max(rep)])
plt.ylim([min(U),(1/2)*W(max(rep))**2*max(rep)**2])
#plt.text(-4,2,'Ω₀ = '+str(w_0)+' s⁻¹ si x>0')
#plt.text(-4,1.8,'Ω₀ = '+str(W_0)+' s⁻¹ si x<0')
plt.savefig('Figura potencial del oscilador.png')
plt.show()

ECUACIÓN DIFERENCIAL

In [ ]:
def EDO(t,y,w):
    x,v = y
    return [v,-2*amort*v-W(x)**2*x+f_0*np.cos(w*t)]

CÁLCULO DE LOS INTERVALOS DE TIEMPO ENTRE CHOQUES SUCESIVOS

In [ ]:
def time_intervals(x):
    v = x.copy()
    bin = v[1:(t.size-1)]
    bin[[v[i]*v[i-1]>0 for i in range(1,t.size-1)]] = 0
    bin[bin<0] = -1
    bin[bin>0] = 1
    v[v.size-1] = -1

    tindpos = np.where(v > 0)[0]
    tindneg = np.where(v < 0)[0]

    return (tindneg-tindpos)[1:(len(tindneg-tindpos)-1)]*dt

CÁLCULO DEL MOVIMIENTO DEL SISTEMA

Integración de la EDO

In [ ]:
pos = []
vel = []
xp = []
vp = []
tdist = []

for i in range(np.size(w)):
    sol = scipy.integrate.solve_ivp(EDO, [0,tf], [x_0,v_0], method = 'RK23',t_eval = t,args = [w[i]])
    x,v = sol.y
    pos.append(x)
    vel.append(v)

    T = 2*pi/w[i] # Periodo fuerza externa
    n = int(tf/T) # Número de puntos del diagrama de Poincaré
    ind = np.array([np.searchsorted(t,T*(i+1)) for i in range(n-1)]) # Calculamos los índices del diagrama de Poincaré
    xp.append(x[ind]) # Asignamos índices encontrados en tiempos con posiciones y velocidades (/max(x[ind]))
    vp.append(v[ind])

    tdist.append(time_intervals(x)) # Calculamos los intervalos de tiempo entre choques

DIMENSIÓN FRACTAL

Método de Box Counting

In [ ]:
def FractalDimension(xp,vp,w): # Función que calcula la dimensión fractal de una imagen
    fig = plt.figure()
    plt.plot(xp[50:],vp[50:],'.',color='red',markersize='1')
    plt.axis('off')
    plt.xlim([min(xp)-0.1,max(xp)])
    plt.ylim([min(vp)-0.1,max(vp)])
    plt.savefig('Poincaré ω = '+str(format(w,'.3f'))+'.png', transparent=True)
    plt.close(fig)
    image = plt.imread('Poincaré ω = '+str(format(w,'.3f'))+'.png')[:,:,3]
    
    Lx = image.shape[0]
    Ly = image.shape[1]
    pixels=[]
    Ns=[]
    E = np.logspace(0, 2, num = 100, endpoint=False, base=2)
    #E = [0.01,0.1,0.5,1,2,4,8,16,32]

    for i in range(Lx):
        for j in range(Ly):
            if image[i,j] > 0:
                pixels.append((i,j))

    pixels=np.array(pixels)
    
    for eps in E:
        Box, ejes = np.histogramdd(pixels, bins=(np.arange(0,Lx,eps),np.arange(0,Ly,eps)))
        Ns.append(np.sum(Box>0))
    fig2 = plt.figure()
    plt.plot(np.log(E),np.log(Ns),'.')
    plt.close(fig2)
    sol = np.polyfit(np.log(E),np.log(Ns),1,full=True)
    dF = -sol[0][0]
    r = sol[1]
    
    if r > 0.2:
        return nan,nan
    else:
        return dF,r

Cálculo de la dimensión fractal

In [ ]:
df = np.zeros(np.size(w))
res = np.zeros(np.size(w))

for i in range(np.size(w)):
    df[i],res[i] = FractalDimension(xp[i],vp[i],w[i])

EVOLUCIÓN DEL DIAGRAMA DE POINCARÉ CON W

Actualización del diagrama en cada frame de la animación

In [ ]:
def update_Poincare(num,xp,vp):
    plt.cla()
    plt.plot(xp[num][50:],vp[num][50:],'.',color='red',markersize='1')
    plt.xlim([-0.05,0.4])
    plt.ylim([-0.25,0.4])
    plt.title('Evolución del diagrama de Poincaré con ω')
    plt.xlabel('x (m)')
    plt.ylabel('v (m/s)')
    plt.suptitle('ω = '+str(format(w[num],'.3f'))+' s⁻¹')

Representación de resultados mediante animación

In [ ]:
im = plt.figure()
fig = plt.plot(xp[0][50:],vp[0][50:],'.',color='red',markersize='1')
plt.xlim([-0.05,0.4])
plt.ylim([-0.25,0.4])
plt.title('Evolución del diagrama de Poincaré con ω')
plt.xlabel('x (m)')
plt.ylabel('v (m/s)')
plt.suptitle('ω = '+str(format(w[0],'.3f'))+' s⁻¹')

animP = animation.FuncAnimation(im, update_Poincare, np.size(w), fargs = (xp,vp, ), interval = 100)
animP.save('Evolución Poincaré ω.gif')
animP.save('Evolución Poincaré ω.mp4')

DIAGRAMA DE POINCARÉ SEGÚN EL INSTANTE DE TIEMPO EN EL QUE SE REGISTREN LOS PUNTOS (INDEPENDIENTE DEL RESTO DEL CÓDIGO)

Solución del movimiento para una frecuencia en el régimen caótico

In [ ]:
wT = 1.411
tfT = 50000
tT = np.linspace(0,tfT,int(tfT/dt)+1)
xpT = []
vpT = []

solT = scipy.integrate.solve_ivp(EDO, [0,tfT], [x_0,v_0], method = 'RK23',t_eval = tT,args = [wT])
xT,vT = solT.y

In [ ]:
TT = 2*pi/wT # Periodo fuerza externa
nT = int(tfT/TT) # Número de puntos del diagrama de Poincaré

for j in range(int(np.round(TT*10))):
    indT = np.array([np.searchsorted(tT,TT*(i+1)+j*0.1) for i in range(nT-1)]) # Calculamos los índices del diagrama de Poincaré
    xpT.append(xT[indT]) # Asignamos índices encontrados en tiempos con posiciones y velocidades (/max(x[ind]))
    vpT.append(vT[indT])

DISTRIBUCIÓN DE INTERVALOS DE TIEMPO

In [ ]:
num_bin = 8000
tdist_hist = np.zeros([w.size,num_bin])

for i in range(w.size):
    tdist_hist[i,:] = np.histogram(tdist[i], bins = num_bin, range = [0,8], density = True)[0]

ALMACENAMIENTO DE DATOS

In [ ]:
fracdim = np.column_stack([df,res,w])
np.savetxt('dimensión_fractal.txt',fracdim)

In [ ]:
for i in range(w.size):
    np.savetxt('xp ω = '+str(format(w[i],'.3f'))+'.txt',xp[i])
    np.savetxt('vp ω = '+str(format(w[i],'.3f'))+'.txt',vp[i])
    np.savetxt('tdist ω = '+str(format(w[i],'.3f'))+'.txt',tdist_hist[i])

In [ ]:
for i in range(int(np.round(TT*10))):
    np.savetxt('xpT T + '+str(format(i*0.1,'.1f'))+'.txt',xpT[i])
    np.savetxt('vpT T + '+str(format(i*0.1,'.1f'))+'.txt',vpT[i])